<a href="https://colab.research.google.com/github/Jam516/MEP-object-detection/blob/master/Thesis_Model_gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Tensorflow

In [0]:
!pip3 install numpy==1.17.4

In [0]:
%tensorflow_version 1.x

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

## Mount drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
%cd '/content/gdrive/My Drive/Thesis/Home'

## Install API

In [0]:
!git clone https://github.com/tensorflow/models.git
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI; make; cp -r pycocotools /content/gdrive/My\ Drive/Thesis/Home/models/research/

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

In [0]:
%cd /content/gdrive/My Drive/Thesis/Home/models/research/
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/Thesis/Home/models/research/:/content/gdrive/My Drive/Thesis/Home/models/research/slim'

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

In [0]:
!python setup.py build
!python setup.py install

## **Setup**
Run at start of every session

In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/Thesis/Home/models/research/:/content/gdrive/My Drive/Thesis/Home/models/research/slim'

In [0]:
%cd /content/gdrive/My Drive/Thesis/Home/models/research/

In [0]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Test setup

In [0]:
!python object_detection/builders/model_builder_test.py

## **Download and Configure Model**

In [0]:
%cd /content/gdrive/My Drive/Thesis/Home/models/research/object_detection

In [0]:
# unpack model after manually uploading it
# !tar -xvf model.tar
!unzip model.zip

Copy in config file

## **Training**

In [0]:
%cd /content/gdrive/My Drive/Thesis/Home/models/research/object_detection

In [0]:
!python model_main.py --logtostderr --model_dir=training/ --pipeline_config_path=training/faster_rcnn_nas_coco.config
# !python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/faster_rcnn_nas_coco.config

## Export Model

In [0]:
%cd /content/gdrive/My Drive/Thesis/Home/models
import os
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/Thesis/Home/models/research/:/content/gdrive/My Drive/Thesis/Home/models/research/slim'
%cd /content/gdrive/My Drive/Thesis/Home/models/research/object_detection

In [0]:
!python export_inference_graph.py \
--input_type image_tensor \
--pipeline_config_path training/faster_rcnn_nas_coco.config \
--trained_checkpoint_prefix training/model.ckpt-7515 \ #change to appropriate checkpoint
--output_directory Exp_graph/

In [0]:
!zip -r MY_exp_g.zip Exp_graph

## **Test Model**

In [0]:
MODEL_NAME = 'My_exp_graph'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = 'training/object-detection.pbtxt'
NUM_CLASSES = 3

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

sys.path.append("..")
from object_detection.utils import ops as utils_ops
%matplotlib inline
from utils import label_map_util
from utils import visualization_utils as vis_util
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
# Detection -------------------------------------------------------

PATH_TO_TEST_IMAGES_DIR = 'test_images/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.png'.format(i)) for i in range(3, 6) ]
# Size, in inches, of the output images.
IMAGE_SIZE = (24, 20) 


def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in ['num_detections', 'detection_boxes', 'detection_scores','detection_classes', 'detection_masks']:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
      if 'detection_masks' in tensor_dict:
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
      # Run inference
      output_dict = sess.run(tensor_dict, feed_dict={image_tensor: image})
      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

x = 1
for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=3)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)
    plt.savefig('pic'+str(x)+'.png')
    x = x + 1

Author: John Kufuor


Adapted From: https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb

https://pythonprogramming.net/introduction-use-tensorflow-object-detection-api-tutorial/

https://medium.com/analytics-vidhya/training-an-object-detection-model-with-tensorflow-api-using-google-colab-4f9a688d5e8b